In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [36]:
data_type = 'sim'
lengths = [90,420]
num_series = (50,300)
structures = ['stationary','trend']
int_type = 'hom'
treatment_rate = 0.3
pred_len = 24
sim_results_path = os.path.join(os.getcwd(),'sim/')

In [11]:
#Load DeepProbCP metrics
deep_met_path = os.path.join(sim_results_path,'DeepProbCP/metrics/')
metrics_cp = {}
for length in lengths:
    for series in num_series:
        for structure in structures:
            data_name = data_type + '_' +str(length)+ '_' +str(series)+ '_' + structure+ '_' +int_type + '_'
            treated_metrics = pd.read_csv(deep_met_path+ 'mean_median_treated_' + data_name + 'LSTMcell_cocob_12_24_without_stl_decomposition.txt', 
                                          header = None, sep = ' ')
            control_metrics = pd.read_csv(deep_met_path +'mean_median_control_' + data_name + 'LSTMcell_cocob_12_24_without_stl_decomposition.txt', 
                                          header = None, sep = ' ')
            agg_smape = np.round((treated_metrics.iloc[0,1] * treatment_rate) + ( control_metrics.iloc[0,1] * (1-treatment_rate) ),3)
            agg_mase = np.round((treated_metrics.iloc[1,1] * treatment_rate) + ( control_metrics.iloc[1,1] * (1-treatment_rate) ),3)
            metrics_cp[data_name] = [agg_smape, agg_mase]

In [16]:
metrics_cp

{'sim_90_50_stationary_hom_': [0.016, 0.231],
 'sim_90_50_trend_hom_': [0.051, 0.562],
 'sim_90_300_stationary_hom_': [0.018, 0.24],
 'sim_90_300_trend_hom_': [0.047, 0.528],
 'sim_420_50_stationary_hom_': [0.018, 0.243],
 'sim_420_50_trend_hom_': [0.027, 0.438],
 'sim_420_300_stationary_hom_': [0.017, 0.239],
 'sim_420_300_trend_hom_': [0.026, 0.429]}

In [12]:
# Load TSMixer metrics
tsmixer_met_path = os.path.join(sim_results_path,'tsmixer/metrics/')
metrics_tsmixer = {}
for length in lengths:
    for series in num_series:
        for structure in structures:
            temp_control = {'smape' : [] , 'mase' : []}
            temp_treated = {'smape': [] , 'mase' : []}
            for iter in range(1,4):
                data_name = data_type + '_' +str(length)+ '_' +str(series)+ '_' + structure+ '_' +int_type + '_'
                control_iter = pd.read_csv(tsmixer_met_path+ 'TSMixer' + '_' + data_type + '_'+ data_name + 'M_sl12_pl24_iter' + \
                                            str(iter) + '_metrics_control.csv')
                treated_iter = pd.read_csv(tsmixer_met_path+ 'TSMixer' + '_' + data_type + '_' + data_name + 'M_sl12_pl24_iter' + str(iter) + \
                                           '_metrics_treated.csv')
                temp_control['smape'].append(control_iter['smape'].values[0])
                temp_control['mase'].append(control_iter['mase'].values[0])
                temp_treated['smape'].append(treated_iter['smape'].values[0])
                temp_treated['mase'].append(treated_iter['mase'].values[0])
            control_metrics = list([np.mean(temp_control['smape']),np.mean(temp_control['mase'])])
            treated_metrics = list([np.mean(temp_treated['smape']),np.mean(temp_treated['mase'])])
            agg_smape = np.round((treated_metrics[0] * treatment_rate) + ( control_metrics[0] * (1-treatment_rate) ),3)
            agg_mase = np.round((treated_metrics[1] * treatment_rate) + ( control_metrics[1] * (1-treatment_rate) ),3)
            metrics_tsmixer[data_name] = [agg_smape, agg_mase]

In [13]:
# Load ASCM metrics
ascm_met_path = os.path.join(sim_results_path,'ascm/metrics/')
metrics_ascm = {}
for length in lengths:
    for series in num_series:
        for structure in structures:
            data_name = data_type + '_' +str(length)+ '_' +str(series)+ '_' + structure+ '_' +int_type + '_'
            treated_metrics = pd.read_csv(ascm_met_path+ data_name + 'metrics_control.txt', header = None, sep = ' ')
            control_metrics = pd.read_csv(ascm_met_path+ data_name + 'metrics_treated.txt', header = None, sep = ' ')
            agg_mase = np.round((treated_metrics.iloc[0,1] * treatment_rate) + ( control_metrics.iloc[0,1] * (1-treatment_rate)),3)
            agg_smape = np.round((treated_metrics.iloc[1,1] * treatment_rate) + ( control_metrics.iloc[1,1] * (1-treatment_rate)),3)
            metrics_ascm[data_name] = [agg_smape, agg_mase]

# Load CausalArima metrics
causar_met_path = os.path.join(sim_results_path,'CausalArima/metrics/')
metrics_causar = {}
for length in lengths:
    for series in num_series:
        for structure in structures:
            data_name = data_type + '_' +str(length)+ '_' +str(series)+ '_' + structure+ '_' +int_type + '_'
            treated_metrics = pd.read_csv(causar_met_path+ data_name + 'metrics_control.txt', header = None, sep = ' ')
            control_metrics = pd.read_csv(causar_met_path+ data_name + 'metrics_treated.txt', header = None, sep = ' ')
            agg_mase = np.round((treated_metrics.iloc[0,1] * treatment_rate) + ( control_metrics.iloc[0,1] * (1-treatment_rate)),3)
            agg_smape = np.round((treated_metrics.iloc[1,1] * treatment_rate) + ( control_metrics.iloc[1,1] * (1-treatment_rate)),3)
            metrics_causar[data_name] = [agg_smape, agg_mase]


In [17]:
df_metrics = pd.DataFrame(index=metrics_cp.keys(), columns=['TSMixer','DeepProbCP','ASCM','CausalArima'])
df_metrics['TSMixer'] = metrics_tsmixer.values()
df_metrics['DeepProbCP'] = metrics_cp.values()
df_metrics['ASCM'] = metrics_ascm.values()
df_metrics['CausalArima'] = metrics_causar.values()

df_metrics

,TSMixer,DeepProbCP,ASCM,CausalArima
sim_90_50_stationary_hom_,"[0.085, 1.214]","[0.016, 0.231]","[0.063, 0.905]","[0.05, 0.737]"
sim_90_50_trend_hom_,"[0.103, 1.138]","[0.051, 0.562]","[0.067, 0.758]","[0.061, 0.682]"
sim_90_300_stationary_hom_,"[0.092, 1.257]","[0.018, 0.24]","[0.067, 0.9]","[0.05, 0.672]"
sim_90_300_trend_hom_,"[0.102, 1.134]","[0.047, 0.528]","[0.068, 0.767]","[0.047, 0.526]"
sim_420_50_stationary_hom_,"[0.025, 0.352]","[0.018, 0.243]","[0.066, 0.902]","[0.037, 0.504]"
sim_420_50_trend_hom_,"[0.025, 0.403]","[0.027, 0.438]","[0.071, 1.175]","[0.027, 0.451]"
sim_420_300_stationary_hom_,"[0.023, 0.322]","[0.017, 0.239]","[0.065, 0.896]","[0.038, 0.522]"
sim_420_300_trend_hom_,"[0.029, 0.478]","[0.026, 0.429]","[0.075, 1.23]","[0.028, 0.459]"


## ATE


In [54]:
# Real ATE
sim_data_path = os.path.join(os.getcwd(),'..', 'data/sim/')
real_ate_dict = {}
for length in lengths:
    for series in num_series:
        for structure in structures:
            data_name = data_type + '_' +str(length)+ '_' +str(series)+ '_' + structure+ '_' +int_type 
            treated_indices = np.loadtxt(sim_data_path + data_name + '_' + 'treated_indices' + '.txt', dtype = int)
            real_data = pd.read_csv(sim_data_path + data_name + '.csv').iloc[length-pred_len:,treated_indices]
            counterfactual_data = pd.read_csv(sim_data_path + data_name + '_' + 'true_counterfactual' '.csv').iloc[length-pred_len:,treated_indices]
            ate = np.mean(counterfactual_data - real_data)
            real_ate_dict[data_name] = ate

In [55]:
real_ate_dict

{'sim_90_50_stationary_hom': 6.7239591011320945,
 'sim_90_50_trend_hom': 13.763941115586693,
 'sim_90_300_stationary_hom': 7.055929241637349,
 'sim_90_300_trend_hom': 13.453976952378618,
 'sim_420_50_stationary_hom': 7.108703015934114,
 'sim_420_50_trend_hom': 24.064092635667805,
 'sim_420_300_stationary_hom': 6.91126835040997,
 'sim_420_300_trend_hom': 24.069603280024772}